## Imports

In [16]:
import googlemaps
import pandas as pd
import numpy as np
import os
import folium
import glob

## Read-in

In [2]:
df = pd.read_csv('multifamily distress - Sheet1.csv')

## Image Link Formatter

In [18]:
image_links = glob.glob('*.png')

In [22]:
# Step 1: Create a mapping from addresses to URLs
# Assuming the address is the filename without the extension
url_mapping = {url.rsplit('.', 1)[0]: url for url in image_links}

# Step 2: Map URLs to DataFrame
df['ImageURL'] = df['address'].map(url_mapping)

## Geocode

In [25]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [26]:
def geocode(add):
    g = gmaps_key.geocode(add)
    lat = g[0]["geometry"]["location"]["lat"]
    lng = g[0]["geometry"]["location"]["lng"]
    return (lat, lng)

df['geocoded'] = df['address'].apply(geocode)

In [27]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].str.strip('()').str.split(', ', expand=True)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

## Correction section

In [6]:
# df.at[index#,'col_name']

## Format Data

In [28]:
### Insert NaNs if needed ###
df = df.replace('N/A', np.nan)

## Prepare Data

In [29]:
df['color'] = ''

df.loc[df['status'] == 'auctioned off', 'color'] = 'red'
df.loc[df['status'] == 'foreclosure filed', 'color'] = 'orange'
df.loc[~df['status'].isin(['auctioned off', 'foreclosure filed']), 'color'] = 'purple'

## HTML Popup Formatter

In [30]:
df.head(1)

,address,status,value,lender,borrower,Unnamed: 5,geocoded,lat,lon,color,legend_text,ImageURL
0,"5200 North Sheridan Road, Chicago 60640",auctioned off,"$40,000,000.00",CMBS,Spirit Invesment/The Bascom Group,(auction value),"(41.97654379999999, -87.6553486)",41.976544,-87.655349,red,auctioned off (red),"5200 North Sheridan Road, Chicago 60640.png"


For the apartments that have the status of "auctioned off" could we change the label from "value" to "auction value" and then for the rest of them change the label from "value" to loan value?

In [41]:
def popup_html(row):
    address = row['address']
    status = row['status']
    value = row['value']
    lender = row['lender']
    borrower = row['borrower']
    image = row['ImageURL']
    
    # Check the status and assign the appropriate label
    value_label = "Auction Value" if status == "auctioned off" else "Loan Value"
    
    # Check if an image URL exists and adjust HTML accordingly
    if pd.notnull(image) and image != '':
        image_html = f'<img src="{image}" width="256" height="156"><br><em>Image Credit: Google Maps</em>'
    else:
        image_html = '<em>No image available</em>'
    
    html = f'''
    <!DOCTYPE html>
    <html>
    {image_html}
    <br>______________________________________<br>
    <strong>Address: </strong>{address}<br>
    <strong>Status: </strong>{status}<br>
    <strong>{value_label}: </strong>{value}<br>
    <strong>Lender: </strong>{lender}<br>
    <strong>Borrower: </strong>{borrower}<br>
    </html>
    '''
    return html

In [39]:
df['legend_text'] = df['status'] + ' (' + df['color'] + ')'

In [40]:
title_html = '''
             <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format('Multi-family distress in Chicago')

### Create map container ###
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=9.5, tiles=None)

# Initialize a dictionary to hold the FeatureGroups for each status
status_groups = {}

for index, row in df.iterrows():
    status = row['legend_text']
    icon_color = row['color']  # Assuming you still want to use color for marker icons
    
    # Check if the FeatureGroup for this status already exists; if not, create it
    if status not in status_groups:
        status_groups[status] = folium.FeatureGroup(name=status).add_to(m)
    
    # Create a Marker for each row
    marker = folium.Marker(
        location=[row['lat'], row['lon']],
        popup=folium.Popup(popup_html(row), max_width=400),
        icon=folium.Icon(color=icon_color)  # Use the 'color' value for the icon
    )
    
    # Add the marker to the corresponding status group
    marker.add_to(status_groups[status])

# Add a LayerControl to allow users to toggle status groups
folium.LayerControl(collapsed=False).add_to(m)
folium.TileLayer('OpenStreetMap',control=False).add_to(m)

m.get_root().html.add_child(folium.Element(title_html))

# Display map
m

In [14]:
m.save('index.html')

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [15]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/chicago_multifamily_distress_02_05_24
